




# Final Assignment
## Deep Learning in Python
#### Group 7: Sudjeeni Bonevacia, Mathilde ter Veen, Gigi Vissers

## Introduction to the notebook

The current computer vision project is concerned with training a convolutional neural network to detect lower-grade glioma brain tumors on MRI-scans. This type of brain tumor, although better manageable than their higher-grade counterparts, occurs often in younger adults and eventually progresses to a fatal stage with an average survival of 7 years after diagnosis (Claus et al., 2016). It is therefore of vital importance that they can be accurately detected from MRI images. Although the field of deep learning is not yet advanced enough to fully replace radiologists at this task, significant progress has been made in the field with classification CNN’s reaching accuracies of over 95% (Irmak, 2021; Li et al., 2021). Skilled radiologists reach accuracies of between 92 and 95% (Yan et al., 2016).

The data used comes from 110 patients who had a lower-grade glioma brain tumor from The Cancer Genome Atlas (TCGA), and consists of images obtained from The Cancer Imaging Archive (TCIA) as well as manually created fluid-attenuated inversion recovery (FLAIR) masks. The FLAIR technique for MRI’s is meant to suppress darkening cerebrospinal fluid effects on the images, which brings out the contrast between the pixels. In total, there are 3929 images in the data set (Buda, Saha & Mazurowski, 2019).

For this taks, we have started with the code of Anant Gupt (https://www.kaggle.com/anantgupt/brain-mri-detection-segmentation-resunet). We added advanced data augmentation techniques such as affine and elastic transforms, and tuned the hyperparameters to achieve higher accuracies.

Besides merely spotting a tumor on an MRI, having algorithms that can identify type of tumor from its shape characteristics seems a viable path for the future as it can help save physicians considerable time for manual labelling, potentially reduces ambiguity when classifying the images, and most importantly increase the likelihood of successful treatment through early detection (Li et al., 2021; Sajjad et al., 2019).

Therefore, as a special twist to our computer vision project, we decided to add a segmentation task on top of the classification task described above. To this end, we apply image segmentation techniques, which cluster the parts of the brain images into tumor or non-tumor areas. This happens at the pixel-level of the images to ensure that localization of the different classes can be realized (Ronneberger, Fischer & Brox, 2015). In order to evaluate the performance of this segmentation task we use the Dice coefficient, a metric of similarity between a predicted segmentation and the ground truth, most often used for this type of task in AI.

The baseline code by Kaggle user Monkira (https://www.kaggle.com/monkira/brain-mri-segmentation-using-unet-keras) that we built on for this task uses a U-net architecture, which is often used in the literature for this type of medical classification problem. In this type of network, based on a fully convolutional neural network, pooling operations are replaced by upsampling operations (Ronneberger et al., 2015).


# 1. IMPORTING LIBRARIES AND DATASET

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import cv2
from skimage import io

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
import tensorflow.keras.backend as K

import plotly.express as px

import random
import glob
from sklearn.preprocessing import StandardScaler, normalize
from IPython.display import display

In [ ]:
!pip install -q kaggle
from google.colab import files

files.upload()

Saving kaggle (1).json to kaggle (1) (1).json


{'kaggle (1) (1).json': b'{"username":"krishthakur351","key":"a4d2e70ba6b869779c79de8f1a4c80f1"}'}

In [ ]:
! mkdir ~/.kaggle  -p
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d goyaladi/fraud-detection-dataset


Dataset URL: https://www.kaggle.com/datasets/goyaladi/fraud-detection-dataset
License(s): CC0-1.0
fraud-detection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


### 1.2 Importing the dataset

In the dataset lgg-mri-segmentation we have the brain images and their corresponding masks (so the tumor location or no tumor) provided in .tif formats. Not every brain image has a corresponding mask, because it doesn't contain a tumor.
In addition, we have a csv file with information about the 110 patients. The .csv file that contains for example age, ethnicity, death and the genomic clusters of patients. We will not be using this information in our analysis. In the graph below you can see that most of the patients are between the age of 30 and 60, so lower-grade glioma brain tumors often arise in young adults (Claus et al., 2015).

In [ ]:
df = pd.read_csv("/content/fraud-detection-dataset")
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/content/fraud-detection-dataset'

In [ ]:
# This shows the first 6 rows of the patient data
patientdata.head()

In [ ]:
# Make objects with the images and masks.
brain_scans = []
mask_files = glob.glob('/content/lgg-mri-segmentation/kaggle_3m/*/*_mask*')

for i in mask_files:
    print(i)
    brain_scans.append(i.replace('_mask',''))

print(brain_scans[:10])
print(mask_files[:10])

### 1.3 Creating the final data frame:

In this section we will combine the brain scans with their corresponding 'masks' in a dataframe. Next, we use a function to determine which masks contain a tumor and which masks are empty. The source for this function is: https://www.kaggle.com/shiveshchowdary/starter-image-segmentation.

Then we apply the function to the masks in the dataframe and add a column with 0's and 1's, **where 0 indicate a mask with no tumor and 1 indicate a mask with a tumor.**
This dataframe is needed for the classification task, because then the model can get the brain scans as input and train the column with the 0's and 1's as output.

In [ ]:
# Make a dataframe with the images and their corresponding masks
data_img = pd.DataFrame({
    "image_path":brain_scans,
    "mask_path":mask_files
})

In [ ]:
# Make a function that search for the largest pixel value in the masks, because that will indicate if the image have
# a corresponding mask with a tumor or not.
def positive_negative_diagnosis(file_masks):
    mask = cv2.imread(file_masks)
    value = np.max(mask)
    if value > 0:
        return 1
    else:
        return 0

# Apply the function to the masks and return back a column with 1 and zeros, where 0 indicate no tumor and 1 a tumor.
data_img["Tumor"] = data_img["mask_path"].apply(lambda x: positive_negative_diagnosis(x))

In [ ]:
data_img # showing the data

# 2. DATA VISUALISATION

In [ ]:
# How many non-tumors (0) and tumors (1) are in the data
data_img['Tumor'].value_counts()

In [ ]:
# Data vizualisation
# SOURCE: https://www.kaggle.com/monkira/brain-mri-segmentation-using-unet-keras#Create-data-frame-and-split-data-on-train-set,-validation-set-and-test-set
# https://www.kaggle.com/saivikassingamsetty/brain-tumor-segmentation-with-unet/notebook

for i in range(1,40, 2):
    img_path=brain_scans[i]
    msk_path=mask_files[i]
    img=cv2.imread(img_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    msk=cv2.imread(msk_path)

    #Plot the Brain MRI scans
    original = img.copy()
    fig, ax = plt.subplots(1,3,figsize = (15,5))
    ax[0].imshow(original)
    ax[0].set_title("Brain MRI")

    # Plot the corresponding mask
    main = original.copy()
    mask = msk.copy()
    ax[1].imshow(mask)
    ax[1].set_title("Tumor Location")

    # Plot the Brain MRI scan with their mask
    main = original.copy()
    label = cv2.imread(msk_path)
    sample = np.array(np.squeeze(label), dtype = np.uint8)
    contours, hier = cv2.findContours(sample[:,:,0],cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    sample_over_gt = cv2.drawContours(main, contours, -1,[255,0,0], thickness=-1)
    ax[2].imshow(sample_over_gt)
    ax[2].set_title("MRI Brain with highlighted Tumor")

# 3. CREATING TEST/TRAIN/VALIDATION SET

In [ ]:
# Convert the data in mask column to string format, to use categorical mode in flow_from_dataframe
data_img['Tumor'] = data_img['Tumor'].apply(lambda x: str(x))
data_img.info()

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_img,test_size = 0.1)
train, val = train_test_split(train,test_size = 0.2)
print(train.values.shape)
print(val.values.shape)
print(test.values.shape)

train.head()
val.head()

### 3.1 Seeing how many tumors are in the train, validation and test set, respectively

In [ ]:
#  SOURCE: adapted from https://www.kaggle.com/anantgupt/brain-mri-detection-segmentation-resunet#2:-DATA-VISUALIZATION

import plotly.graph_objects as go  # using plotly to create interactive plots


fig = go.Figure([go.Bar(x=train['Tumor'].value_counts().index,
                        y=train['Tumor'].value_counts(),
                        width=[.4, .4],
                       )
                ])
fig.update_traces(marker_color=['darkolivegreen', 'firebrick'], opacity = 0.7
                 )

fig.update_layout(title_text="Tumor Count Train Set",
                  width=700,
                  height=550,
                  yaxis=dict(
                             title_text="Count",
                             tickmode="array",
                             titlefont=dict(size=20)
                           )
                 )

fig.update_yaxes(range = list([0,2000]))
fig.update_xaxes(tick0 = 0, dtick = 1)

fig.show()

fig2 = go.Figure([go.Bar(x=val['Tumor'].value_counts().index,
                        y=val['Tumor'].value_counts(),
                        width=[.4, .4]
                       )
                ])
fig2.update_traces(marker_color=['darkolivegreen', 'firebrick'], opacity = 0.7
                 )
fig2.update_layout(title_text="Tumor Count Validation Set",
                  width=700,
                  height=550,
                  yaxis=dict(
                             title_text="Count",
                             tickmode="array",
                             titlefont=dict(size=20)
                           )
                 )

fig2.update_yaxes(range = list([0,2000]))
fig2.update_xaxes(tick0 = 0, dtick = 1)


fig2.show()

fig3 = go.Figure([go.Bar(x=test['Tumor'].value_counts().index,
                        y=test['Tumor'].value_counts(),
                        width=[.4, .4]
                       )
                ])
fig3.update_traces(marker_color=['darkolivegreen', 'firebrick'], opacity = 0.7
                 )
fig3.update_layout(title_text="Tumor Count Test Set",
                  width=700,
                  height=550,
                  yaxis=dict(
                             title_text="Count",
                             tickmode="array",
                             titlefont=dict(size=20)
                           )
                 )

fig3.update_yaxes(range = list([0,2000]))
fig3.update_xaxes(tick0 = 0, dtick = 1)

fig3.show()

# 4. CLASSIFICATION MODEL TO DETECT EXISTENCE TUMOR

In this part of the notebook a convolutional neural network (CNN) approach will be introduced, including Data Augmentation to categorize brain MRI scan images into the existence of a tumor and not existence of a tumor.

In some literature it was stated to add Canny Edge Detection to the process. We tried to do this ourselves, but in the way we did it (using cv2.Canny()), it didn't add something to the result.

Transfer learning is used in most of the former build classification models (Kahn et al.,2020; Lundervold & Lundervold, 2019) and different pre-trained models are compared. In most cases the VGG-16 model gave the best results in the sense of accuracy. Expected accuracy based on literature could be somewhere between 92% and 96% (Simonyan & Zisserman, 2014; Kahn et al.,2020).

The classification model can be summarized in the following picture, which is based on the process that is pictured in Kahn et al. (2020). The changes and the add ons that we made in the process are highlighted in the figure below and are further explained in the concerning steps:


![Classificatiemodel.jpg](attachment:25b93b02-64c9-45d1-a48e-6ad68efd47ff.jpg)


### 4.1 Batch size

In this step the batch size is defined. We did the tuning of the batchsize by trying out different batchsizes in the earlier stages of the coding process, before adding the data augmentation.

The tuning is summarized in the following table:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#Table
fig, ax =plt.subplots(1,1)
data=[[0.850, 0.919,  0.896]]
column_labels=['Batchsize 16', 'Batchsize 32', 'Batchsize 64']
row_label = ['Accuracy']
ax.axis('tight')
ax.axis('off')
ax.table(cellText=data,colLabels=column_labels,rowLabels=row_label, loc="center")

plt.show()

#Barplot
# Make a random dataset:
height = [0.850, 0.919, 0.896]
bars = ('16', '32', '64')
x_pos = np.arange(len(bars))

# Create bars and choose color
plt.bar(x_pos, height, color = ['darkseagreen', 'seagreen', 'mediumseagreen'])

# Add title and axis names
plt.title('Accuracy per batchsize')
plt.xlabel('Batchsize')
plt.ylabel('Accuracy')

# Create names on the x axis
plt.xticks(x_pos, bars)

# Show graph
plt.show()

Based on this results we choose a batchsize of 32 because it gave the highest accuracy.

A batchsize of 128 was too big and gave errors. Too large of a batch size will lead to poor generalization. Lowering the learning rate and decreasing the batch size will allow the network to train better, especially in the case of fine-tuning (Kandel & Castelli, 2020).

### 4.2 Data augmentation
A major challenge in brain tumor classification is a lack of available or useable data (Sajjad et al., 2019). Training deep neural networks often requires a vast amount of data to yield accurate classifications, which is why we have opted to include data augmentation to train on. In the area of computer vision, such data augmentation is a very common regularization technique used to combat overfitting on the training dataset, and hence increase performance on other data sets.

Based on Nalepa, Marcinkiewicz and Kawulok (2019), we have elected several data augmentation techniques that have most often and most succesfully been used in similar tumor classification and segmentation tasks on the BraTS18 data set, which has been dubbed as a "standard benchmark for validating existing and emerging brain-tumor detection and segmentation techniques".

We used affine and elastic transformations. With affine transformations, images are flipped, cropped, rotated, and zoomed to produce more images. They are often used in the literature, and can lead to significant increase training images.
**Note:** This type of transform may lead to highly correlated images or anatomically incorrect images, so we have picked only those that we thought would decrease this possibility.
- flipping (horizontal, because the left and right lobe of a brain are symmetrical looking),
-  changes in width, height, and a sublte zoom, because not all brains on the MRI’s are of the same size, and height and width adjustments of the positions of the brain on the scan, so the model learns location invariance.

Then elastic transformationsm, used for shape variations, are often mentioned in the literature as viable data augmentation technique in the medical field to learn context invariance when looking for abnormalities. It draws a gridline over images, and distorts the image along those lines (Nalepa et al., 2019; Isensee et al., 2018).


We have used a function used in two notebooks (and many many online resources!) cited below, and tuned its parameters.

In [ ]:
# SOURCE: https://www.kaggle.com/babbler/mnist-data-augmentation-with-elastic-distortion
# https://www.kaggle.com/bguberfain/elastic-transform-for-data-augmentation

from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates


def elastic_transform(image, alpha_range, sigma, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.

   # Arguments
       image: Numpy array with shape (height, width, channels).
       alpha_range: Float for fixed value or [lower, upper] for random value from uniform distribution.
           Controls intensity of deformation.
       sigma: Float, sigma of gaussian filter that smooths the displacement fields.
       random_state: `numpy.random.RandomState` object for generating displacement fields.
    """

    if random_state is None:
        random_state = np.random.RandomState(None)

    if np.isscalar(alpha_range):
        alpha = alpha_range
    else:
        alpha = np.random.uniform(low=alpha_range[0], high=alpha_range[1])

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha

    x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]), indexing='ij')
    indices = np.reshape(x+dx, (-1, 1)), np.reshape(y+dy, (-1, 1)), np.reshape(z, (-1, 1))

    transformed_images = map_coordinates(image, indices, order=1, mode='reflect').reshape(shape)

    return transformed_images

### 4.7 Evaluation of data augmentation settings

We tuned the augmentation settings to try and find the highest accuracy whilst not overfitting too much.
To this end, we have tried the following settings:


*No augmentation*

Test accuracy: 0.8830

Test loss: 0.2641

*All augmentation standard* (As often used in other notebooks and literature)

Affine transforms:
- horizontal flips = True
- width shift = 0.2
- height shift = 0.2
- zoom range = [0.5, 0.75]

Elastic transforms:
- alpha_range = 8
- sigma = 3

Test accuracy: 0.7917

Test loss: 0.4504

*All augmentations light*

Affine transforms:
- horizontal flips = True
- width shift = 0.01
- height shift = 0.01
- zoom range [0.01,0.25]

Elastic transforms:
- alpha_range: 0.25
- sigma = 0.05

Test accuracy: 0.6132

Test loss: 0.8652

*Only Affine augmentations standard*

Affine transforms:
- horizontal flips = True
- width shift = 0.2
- height shift = 0.2
- zoom range [0.5,0.75]

Elastic transforms:
- alpha_range: -
- sigma = -

Test accuracy: 0.7631

Test loss: 0.4492

*HF and Elastic augmentation*

Affine transforms:
- horizontal flips = True
- width shift = -
- height shift = -
- zoom range = -

Elastic transforms:
- alpha_range = 5
- sigma = 2

Test accuracy: 0.9005

Test loss: 0.2381

*HF and Elastic augmentation 2*

Affine transforms:
- horizontal flips = True
- width shift = -
- height shift = -
- zoom range = -

Elastic transforms:
- alpha_range = 0.5
- sigma = 0.05

Test accuracy: 0.8957

Test loss: 0.2797

*HF and Elastic augmentation 3*

Affine transforms:
- horizontal flips = True
- width shift = -
- height shift = -
- zoom range = -

Elastic transforms:
- alpha_range = [2,6]
- sigma = 2.5

Test accuracy: 0.9008

Test loss: 0.2563

*Just HF*
Affine transforms:
- horizontal flips = True
- width shift = -
- height shift = -
- zoom range = -

Elastic transforms:
- alpha_range = -
- sigma = -

Test accuracy: 0.8193

Test loss: 0.2848


#### Conclusion:
Using all augmentation techniques described before yielded rather low accuracies and high losses. The augmentation techniques that gave the best accuracies compared to the model trained on un-augmented data were horizontal flips and elastic transformations.
This could be due to a number of reasons, including that tumors could occur on both hemispheres of the brain such that horizontal flips did not create images that the model was unlikley to recognize.
The other affine transforms (zooms and shifts), although widely used in the literature, resulted in lower test accuracies. Cropping the image by zooming in might not have worked because a tumor is unlikely to be in the middle of the MRI, thus zooming in on this part does not teach the model much extra.

### Visualising the tuning

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Accuracies: ###########################
height = [0.8830, 0.7917, 0.6132, 0.7631, 0.9005, 0.8957, 0.9008, 0.8193]
bars = ('No Augmentation', 'Full Augmentation standard values', 'Full Augmentation light values', 'Only Affine',
        'Horizontal Flip + Elastic standard', 'Horizontal Flip + Elastic light', 'Horizontal flip + elastic range',
        'Horizontal Flip'
       )

x_pos = np.arange(len(bars))

colours = sns.color_palette("YlGnBu", 8)

# Create bars and choose color
plt.bar(x_pos, height, color = colours)

# Add title and axis names
plt.title('Accuracy per Augmentation Setting')
plt.ylabel('Accuracy')

# Create names on the x axis
plt.yticks((0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0))
plt.xticks(x_pos, bars, rotation = 90)
plt.tick_params(top='off', bottom='on', left='on', right='off', labelleft='on', labelbottom='on')

# Show graph
plt.show()

# Losses #########################
# Accuracies:
height = [0.2641, 0.4504, 0.8652, 0.4492, 0.2381, 0.2797, 0.2563, 0.2848]
bars = ('No Augmentation', 'Full Augmentation standard values', 'Full Augmentation light values', 'Only Affine',
        'Horizontal Flip + Elastic standard', 'Horizontal Flip + Elastic light', 'Horizontal flip + elastic range',
        'Horizontal Flip'
       )

colours = sns.color_palette("YlGnBu", 8)

# Create bars and choose color
plt.bar(x_pos, height, color = colours)

# Add title and axis names
plt.title('Loss per Augmentation Setting')
plt.ylabel('Loss')

# Create names on the x axis
plt.yticks((0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0))
plt.xticks(x_pos, bars, rotation = 90)
plt.tick_params(top='off', bottom='on', left='on', right='off', labelleft='on', labelbottom='on')

# Show graph
plt.show()

### Adding the data augmentation to the image data generator

In [ ]:
!pip install  keras-preprocessing

In [ ]:
from keras_preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(rescale=1./255.,
                             validation_split=0.15,
                             horizontal_flip = True,
                            # zoom_range = ,
                            # width_shift_range= ,
                            # height_shift_range= ,
                             preprocessing_function = lambda x: elastic_transform(x,
                                                                                 alpha_range =  [2,6],
                                                                                 sigma = 2.5)
                            )


train_generator = datagen.flow_from_dataframe(train,
                                              directory='./',
                                              x_col='image_path',
                                              y_col='Tumor',
                                              subset='training',
                                              class_mode='binary',
                                              batch_size=32,
                                              shuffle=True,
                                              target_size=(256,256)
                                             )

valid_generator = datagen.flow_from_dataframe(val,
                                              directory='./',
                                              x_col='image_path',
                                              y_col='Tumor',
                                              subset='validation',
                                              class_mode='binary',
                                              batch_size=32,
                                              shuffle=True,
                                              target_size=(256,256)
                                             )

test_datagen = ImageDataGenerator(rescale=1./255.)

test_generator = test_datagen.flow_from_dataframe(test,
                                                  directory='./',
                                                  x_col='image_path',
                                                  y_col='Tumor',
                                                  class_mode='binary',
                                                  batch_size=32,
                                                  shuffle=False,
                                                  target_size=(256,256)
                                                 )

### 4.3 Define pretrained base

As noted down above, the VGG-16 model as pretrained base, should give us the best results based on literature. The pretrained base is loaded below.


In [ ]:
# Taking a pretrained base
from tensorflow.keras.applications import VGG16

pretrained_base = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(256,256,3)))
pretrained_base.summary()

# The pretrained database not trainable.
pretrained_base.trainable = False

### 4.4 Attach head

In this section we define the classifier head and attach this to the pretrained base.

In the classifier head, the Flatten layer transforms the two dimensional outputs of the base into the one dimensional inputs needed by the head.

The drop out layer omits some of the neurons at each step from the layer making the neurons more independent from the neighbouring neurons. It helps in avoiding overfitting. We tried different drop-out parameters, between 0.2 and 0.5, but the parameters as used in our example code were given the best results.

The Dense layer is the output layer which classifies the image into 1 of the 2 possible classes. The sigmoid function is used for the two-class logistic regression, instead softmax function as was used in the notebook that provided us the example code. The softmax function should be used for the multiclass logistic regression.

The classifier head is based on the accuracy results in other notebooks:

https://www.kaggle.com/anantgupt/brain-mri-detection-segmentation-resunet#7:-CLASSIFIACTION-MODEL-EVALUATION
https://www.kaggle.com/jaykumar1607/brain-tumor-mri-classification-tensorflow-cnn

In [ ]:
# Attaching head
from tensorflow.keras import layers
#from tensorflow.keras.layers.experimental import preprocessing

model = keras.Sequential([

    # Base
    pretrained_base,

    # Head
    layers.Flatten(name='Flatten'),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid'),
])

### 4.5 Train

In defining the loss function, all of the example notebooks that we used, use the categorical_crossentropy, instead of the binary_crossentropy. This strange, because we only have two target classes. With the original code out of that notebooks, we ran the code with the categorical_crossentropy and it gave a much higher accuracy, than with the use of the binary_crossentropy (0.898 compared to 0.628).

This was peculiar because the formulas for both are not so much different:
![download (1).png](attachment:9784585a-959b-4f55-a6c8-d66dd7970eeb.png)

Source: https://newbedev.com/why-binary-crossentropy-and-categorical-crossentropy-give-different-performances-for-the-same-problem.

The solution of this problem is in the coding of the head. The wrong activation function was used. Instead of the softmax-function, the sigmoid function needed to be used with a scalar target and not a one-hot encoded target.

As an optimizer Adam is used. Adam is an SGD algorithm that has an adaptive learning rate that makes it suitable for most problems without any parameter tuning. Adam is a great general-purpose optimizer and we found it used in all sources we found.


In [ ]:
model.compile(loss = 'binary_crossentropy',
              optimizer='adam',
              metrics= ['accuracy']
             )

model.summary()

We used the following callbacks:

EarlyStopping helps us to stop the training of the model early if there is no increase in the parameter. It is a form of regulation to avoid overfitting. We have tried different minimal delta's in the script, between 0.001 and 0.0001. Also looking at the performance of the script, 0.001 gave us the best results.

ModelCheckpoint helps us to save the model by monitoring a specific parameter of the model. In this case, We monitoring validation accuracy by passing val_acc to ModelCheckpoint.

ReduceLROnPlateau helps us reduce learning rate when a metric has stopped improving. Models often benefit from reducing the learning rate by a factor of 2-10 once learning stagnates. This callback monitors a quantity and if no improvement is seen for a 'patience' number of epochs, the learning rate is reduced (https://keras.io/api/callbacks/reduce_lr_on_plateau/).

In [ ]:
earlystopping = EarlyStopping(monitor='val_loss',
                              min_delta=0.001,
                              mode='min',
                              verbose=1,
                              patience=20,
                              restore_best_weights = True
                             )

checkpointer = ModelCheckpoint(filepath="vgg16_1.h5",
                               monitor = 'val_accuracy',
                               verbose=1,
                               save_best_only=True
                              )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='min',
                              verbose=1,
                              patience=15,
                              min_delta=0.0001,
                              factor=0.2
                             )

callbacks = [checkpointer, earlystopping, reduce_lr]

In [ ]:
history = model.fit(train_generator,
              steps_per_epoch= train_generator.n // train_generator.batch_size,
              epochs = 5,
              validation_data= valid_generator,
              validation_steps= valid_generator.n // valid_generator.batch_size,
              callbacks=callbacks)

### 4.6 Classification Model Evaluation

In [ ]:
history.history.keys()

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss']);
plt.plot(history.history['val_loss']);
plt.title("Classification Model LOSS");
plt.ylabel("loss");
plt.xlabel("Epochs");
plt.legend(['train', 'val']);

plt.subplot(1,2,2)
plt.plot(history.history['accuracy']);
plt.plot(history.history['val_accuracy']);
plt.title("Classification Model Acc");
plt.ylabel("Accuracy");
plt.xlabel("Epochs");
plt.legend(['train', 'val']);

### Test accuracy:

In [ ]:
_, acc = model.evaluate(test_generator)
print("Test accuracy : {} %".format(acc*100))


### Literature
Buda, M., Saha, A., & Mazurowski, M. A. (2019). Association of genomic subtypes of lower-grade gliomas with shape features automatically extracted by a deep learning algorithm. *Computers in biology and medicine*, 109, 218-225.

Claus, E. B., Walsh, K. M., Wiencke, J. K., Molinaro, A. M., Wiemels, J. L., Schildkraut, J. M., ... & Wrensch, M. (2015). Survival and low-grade glioma: the emergence of genetic information. Neurosurgical focus, 38(1), E6.

Irmak, E. (2021). Multi-Classification of Brain Tumor MRI Images Using Deep Convolutional Neural Network with Fully Optimized Framework. Iranian Journal of Science and Technology, Transactions of Electrical Engineering, 1-22.

Isensee, F., Kickingereder, P., Wick, W., Bendszus, M., & Maier-Hein, K. H. (2018). No new-net. In International MICCAI Brainlesion Workshop (pp. 234-244). Springer, Cham.

Kandel, I., Castelli, M. (2020). The effect of batch size on the generalizability of the convolutional neural networks on a histopathology dataset, ICT Express, Volume 6, Issue 4, 312-315. https://doi.org/10.1016/j.icte.2020.04.010.

Li, CC., Wu, MY., Sun, YC. (2021). Ensemble classification and segmentation for intracranial metastatic tumors on MRI images based on 2D U-nets. Sci Rep 11, 20634. https://doi.org/10.1038/s41598-021-99984-5

Lundervold, A. S., & Lundervold, A. (2019). An overview of deep learning in medical imaging focusing on MRI. Zeitschrift für Medizinische Physik, 29(2), 102-127. https://doi.org/10.1016/j.zemedi.2018.11.002

Nalepa, J., Marcinkiewicz, M., & Kawulok, M. (2019). Data augmentation for brain-tumor segmentation: a review. Frontiers in computational neuroscience, 13, 83.

Ronneberger, O., Fischer, P., & Brox, T. (2015, October). U-net: Convolutional networks for biomedical image segmentation. In International Conference on Medical image computing and computer-assisted intervention (pp. 234-241). Springer, Cham.

Sajjad, M., Khan, S., Muhammad, K., Wu, W., Ullah, A., & Baik, S. W. (2019). Multi-grade brain tumor classification using deep CNN with extensive data augmentation. Journal of computational science, 30, 174-182.

Seetha, J., & Raja, S. S. (2018). Brain tumor classification using convolutional neural networks. Biomedical & Pharmacology Journal, 11(3), 1457.

Simonyan,  K.,  Zisserman,  K.  (2014).  Very  deep convolutional  networks for  large-scale  image recognition.  CoRR, vol. abs/1409.1556.  